# Firesmoke Data Conversion to IDX using OpenVisus

## Import necessary libraries, install them if you do not have them. This was developed in Python 3.9

In [19]:
# Used to read/manipulate netCDF data
import xarray as xr

# Used to convert to .idx
from OpenVisus import *

# Used for numerical work
import numpy as np

# Used for processing netCDF time data
import datetime

# Used for interacting with OS file system (to get directory file names)
import os

# To load/save final sequence array to file
import pickle

# Used for resampling arrays to fit the same lat/lon grid
from scipy.interpolate import griddata

# for plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# for checking and using timestamps
import pandas as pd

# Accessory, used to generate progress bar for running for loops
# from tqdm.notebook import tqdm
# import ipywidgets
# import jupyterlab_widgets
from tqdm import tqdm

## Get relevant directory paths

In [20]:
# ******* THIS IS WHEN RUNNING FROM LOCAL MACHINE (canada1) **************
# path to all original netCDF files from UBC
firesmoke_dir = "/opt/wired-data/firesmoke/final_union_set"

# path to save idx file and data
idx_dir = "/opt/wired-data/firesmoke/idx"

## Gather information about the metadata of our files, since it is inconsistent file to file. We need to know what values to keep consistent across all files.
<h3 style="white-space: pre-line;">
1. Count number of files there are per firesmoke directory.
2. Determine maximum row,col dimension sizes for pm25 array.
3. Determine maximum latitude longitude grid parameters.
</h3>

In [21]:
# List of all files that are available from UBC
successful_files = np.array([])

# Track all unique sizes of netCDF variables across all files
# creating a dict of variables for each file that has differing attributes
all_unique_dims = {}

# get list of netcdf file names, in alphabetical order
file_names = sorted(os.listdir(firesmoke_dir[0:100]))

# all_unique_dims starts empty
all_unique_dims_empty = 1

# try opening each file, process only if it successfully opens
for file in tqdm(file_names[0:200]):
    # get file's path
    path = f'{firesmoke_dir}/{file}'
    
    # keep track of which files successfully open
    try:
        # open the file with xarray
        ds = xr.open_dataset(path)

        # append file name to successful_files
        successful_files = np.append(successful_files,file)

        # populate all_unique_attr dict upon first successful file opening
        if all_unique_dims_empty:
            all_unique_dims[0] = [ds.sizes, ds.attrs]
            all_unique_dims_empty = 0

        # check if this file has unique attrs different from what we've already tracked
        need_new_key = 1
        for unique_key in all_unique_dims.keys():
            if ds.sizes == all_unique_dims[unique_key][0]:
                # we've already recorded this unique size
                need_new_key = 0
                continue 

        # add a new entry for new size
        if need_new_key:    
            new_key = len(all_unique_dims.keys())
            all_unique_dims[new_key] = [ds.sizes, ds.attrs]
        
    except:
        # netcdf file does not exist
        continue

# Sort list of successful files so they're in order of date
successful_files = np.sort(successful_files).tolist()

100%|██████████| 200/200 [00:01<00:00, 190.38it/s]


### Create resampling grids to resample values on smaller grid to larger grid during conversion to IDX.
By visual inspection of `all_unique_dims` we see there is only 2 unique grid sizes used, we will refer to these as `max_grid` and `min_grid`.

In [26]:
# Set max_grid and min_grid to unwrapped and merged ds.sizes and ds.attr dicts
last_key = len(all_unique_dims.keys())-1
max_grid = {**dict(all_unique_dims[1][0]), **dict(all_unique_dims[1][1])}
min_grid = {**dict(all_unique_dims[last_key][0]), **dict(all_unique_dims[last_key][1])}

# swap if they're incorrectly set
if max_grid['ROW'] * max_grid['COL'] < min_grid['ROW'] * min_grid['COL']:
    tmp = min_grid
    min_grid = max_grid
    max_grid = tmp

# get arrays of bigger lat/lon grid
big_lon = np.linspace(max_grid['XORIG'], max_grid['XORIG'] + max_grid['XCELL'] * (max_grid['COL'] - 1), max_grid['COL'])
big_lat = np.linspace(max_grid['YORIG'], max_grid['YORIG'] + max_grid['YCELL'] * (max_grid['ROW'] - 1), max_grid['ROW'])

# get coordinates made of new lat/lon arrays
big_lon_pts, big_lat_pts = np.meshgrid(big_lon, big_lat)
big_tups = np.array([tup for tup in zip(big_lon_pts.flatten(), big_lat_pts.flatten())])

# get arrays of smaller lat/lon grid
sml_lon = np.linspace(min_grid['XORIG'], min_grid['XORIG'] + min_grid['XCELL'] * (min_grid['COL'] - 1), min_grid['COL'])
sml_lat = np.linspace(min_grid['YORIG'], min_grid['YORIG'] + min_grid['YCELL'] * (min_grid['ROW'] - 1), min_grid['ROW'])

# get coordinates made of small lat/lon arrays
sml_lon_pts, sml_lat_pts = np.meshgrid(sml_lon, sml_lat)
sml_tups = np.array([tup for tup in zip(sml_lon_pts.flatten(), sml_lat_pts.flatten())])

## TESTING `resample_array` AND SCRIBBLES

### This is plotting the oiginal 381x1041 file

In [ ]:
# # Get the PM25 values, squeeze out empty axis
# vals = np.squeeze(sml_ds['PM25'].values)

# # Perform the interpolation
# arr = griddata(sml_tups, vals[15].flatten(), big_tups, method='cubic', fill_value=0)

# # Any values that are less than a given threshold, make it 0
# arr[arr < 1e-15] = 0

# # Reshape the result to match the new grid shape
# arr = arr.reshape((len(big_lat), len(big_lon)))

# arr = arr.astype(np.float32)

In [ ]:
# np.min(arr)

In [ ]:
# type(arr[0,0])

In [ ]:
# # Let's use matplotlib's imshow, since our data is on a grid
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html

# # Initialize a figure and plot, so we can customize figure and plot of data
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
# # ref: https://scitools.org.uk/cartopy/docs/latest/getting_started/index.html
# my_fig, my_plt = plt.subplots(figsize=(15, 6), subplot_kw=dict(projection=ccrs.PlateCarree()))

# # Let's set some parameters to get the visualization we want
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html

# # color PM25 values on a log scale, since values are small
# my_norm = "log" 
# # this will number our x and y axes based on the longitude latitude range
# my_extent = [np.min(sml_lon), np.max(sml_lon), np.min(sml_lat), np.max(sml_lat)]
# # ensure the aspect ratio of our plot fits all data, matplotlib can does this automatically
# my_aspect = 'auto'
# # tell matplotlib, our origin is the lower-left corner
# my_origin = 'lower'
# # select a colormap for our plot and the color bar on the right
# my_cmap = 'viridis'

# # create our plot using imshow
# plot = my_plt.imshow(arr, norm=my_norm, extent=my_extent, 
#           aspect=my_aspect, origin=my_origin, cmap=my_cmap)

# # draw coastlines
# my_plt.coastlines()

# # draw latitude longitude lines
# # ref: https://scitools.org.uk/cartopy/docs/latest/gallery/gridlines_and_labels/gridliner.html
# my_plt.gridlines(draw_labels=True)

# # add a colorbar to our figure, based on the plot we just made above
# my_fig.colorbar(plot,location='right', label='ug/m^3')

# # # Set x and y axis labels on our ax
# # my_plt.set_xlabel('Longitude')
# # my_plt.set_ylabel('Latitude')

# # Set title of our figure
# my_fig.suptitle('Ground level concentration of PM2.5 microns and smaller')

# # # Set title of our plot as the timestamp of our data
# # my_plt.set_title(f'{my_timestamp}')

# # Show the resulting visualization
# plt.show()

### This is visualizing the resampled version of array above, from 381x1041 -> 381x1081 grid

In [ ]:
# # Let's use matplotlib's imshow, since our data is on a grid
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html

# # Initialize a figure and plot, so we can customize figure and plot of data
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
# # ref: https://scitools.org.uk/cartopy/docs/latest/getting_started/index.html
# my_fig, my_plt = plt.subplots(figsize=(15, 6), subplot_kw=dict(projection=ccrs.PlateCarree()))

# # Let's set some parameters to get the visualization we want
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html

# # color PM25 values on a log scale, since values are small
# my_norm = "log" 
# # this will number our x and y axes based on the longitude latitude range
# my_extent = [np.min(big_lon), np.max(big_lon), np.min(big_lat), np.max(big_lat)]
# # ensure the aspect ratio of our plot fits all data, matplotlib can does this automatically
# my_aspect = 'auto'
# # tell matplotlib, our origin is the lower-left corner
# my_origin = 'lower'
# # select a colormap for our plot and the color bar on the right
# my_cmap = 'viridis'

# # create our plot using imshow
# plot = my_plt.imshow(arr_resamp, norm=my_norm, extent=my_extent, 
#           aspect=my_aspect, origin=my_origin, cmap=my_cmap, vmin=.00001, vmax=1)

# # draw coastlines
# my_plt.coastlines()

# # draw latitude longitude lines
# # ref: https://scitools.org.uk/cartopy/docs/latest/gallery/gridlines_and_labels/gridliner.html
# my_plt.gridlines(draw_labels=True)

# # add a colorbar to our figure, based on the plot we just made above
# my_fig.colorbar(plot,location='right', label='ug/m^3')

# # # Set x and y axis labels on our ax
# # my_plt.set_xlabel('Longitude')
# # my_plt.set_ylabel('Latitude')

# # Set title of our figure
# my_fig.suptitle('Ground level concentration of PM2.5 microns and smaller')

# # # Set title of our plot as the timestamp of our data
# # my_plt.set_title(f'{my_timestamp}')

# # Show the resulting visualization
# plt.show()

## Determine sequence of files to load later for IDX conversion

### First determine what hours are available in all datasets, from there we construct final sequence

In [27]:
# for parsing time flags (TFLAG) from netcdf files
# return datetime object of tflag
# param [int, int] tflag: the TFLAG to parse and return as datetime object 
def parse_tflag(tflag):
    year = int(tflag[0] // 1000)
    day_of_year = int(tflag[0] % 1000)
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=day_of_year - 1)

    time_in_day = int(tflag[1])
    hours = time_in_day // 10000
    minutes = (time_in_day % 10000) // 100
    seconds = time_in_day % 100

    full_datetime = datetime.datetime(year, date.month, date.day, hours, minutes, seconds)
    return full_datetime

In [28]:
# get set of all available hours from successful_files
# we use a dictionary so we can index by CDATE
available_dates = {np.int32(file.split('_')[1]): {} for file in successful_files}

rows = []

for file in tqdm(successful_files):
    # get file's path
    path = f'{firesmoke_dir}/{file}'
    
    # open the file with xarray
    ds = xr.open_dataset(path)

    # for each CDATE_CTIME, store their respective TFLAGs
    cdatetime = pd.to_datetime(f"{ds.CDATE}_{ds.CTIME}", format='%Y%j_%H%M%S')
    tflags = ds['TFLAG'][:, 0, :].values

    # append new row of CDATETIMEs with their respective TFLAGs
    rows.append({
            'CDATETIME': cdatetime,
            'TFLAG': tflags
        })
available_dates_df = pd.DataFrame(rows)

100%|██████████| 200/200 [00:00<00:00, 282.29it/s]


In [29]:
available_dates_df

,CDATETIME,TFLAG
0,2021-02-28 02:50:55,"[[2021059, 30000], [2021059, 40000], [2021059,..."
1,2021-02-28 12:00:47,"[[2021059, 90000], [2021059, 100000], [2021059..."
2,2021-02-28 14:49:58,"[[2021059, 150000], [2021059, 160000], [202105..."
3,2021-02-28 20:52:38,"[[2021059, 210000], [2021059, 220000], [202105..."
4,2021-03-01 02:52:15,"[[2021060, 30000], [2021060, 40000], [2021060,..."
...,...,...
195,2021-04-20 03:07:46,"[[2021110, 30000], [2021110, 40000], [2021110,..."
196,2021-04-20 11:31:58,"[[2021110, 90000], [2021110, 100000], [2021110..."
197,2021-04-20 15:15:08,"[[2021110, 150000], [2021110, 160000], [202111..."
198,2021-04-20 21:07:47,"[[2021110, 210000], [2021110, 220000], [202111..."


Step through all hours we want to represent (earliest available firesmoke file's CDATE to present day) and grab from according dispersion_[CDATE]_[CTIME].nc file.

In [30]:
def update_idx_calls(arr, cdatetime, tstep_idx):
    '''
    For the given array, append arguments specifying which dispersion.nc file to open
        and which TFLAG to use as found in dates_df
    :param list arr: array that holds final idx write sequence
    :param str cdatetime: datetime object of CDATE + CTIME of the dispersion file we will open
    :param datetime time_idx: The index of the TFLAG in TFLAG array we will select from dispersion file
    '''
    # get path of file we will use
    cdate_str = cdatetime.date().strftime("%Y%j")
    ctime_str = cdatetime.time().strftime("%H%M%S")
    file_str = f"dispersion_{cdate_str}_{ctime_str}.nc"
    path = f'{firesmoke_dir}/{file_str}'

    # open the file with xarray
    ds = xr.open_dataset(path)
    arr.append([file_str, parse_tflag(ds['TFLAG'].values[tstep_idx][0]), tstep_idx])

    return arr

## **TODO**: Make sure the `cdate` and `ctime` you're selecting is actually correct... as shown below for example, can a file 'made in the future' give the latest forecast prediction for a timestep 'in the past'?

In [31]:
# Arrays to hold the final order we will index files
from datetime import timedelta

idx_calls = []

# Define the start and end dates we will step through
# start_date = datetime.datetime.strptime("2021059", "%Y%j")
# end_date = datetime.datetime.strptime("2025317", "%Y%j")

# TMP DATES
start_date = datetime.datetime.strptime("2021059", "%Y%j")
end_date = datetime.datetime.strptime("2021060", "%Y%j")

# iterate over each day
current_date = start_date
# iterate over each hour of the current day
current_hour = datetime.datetime(current_date.year, current_date.month, current_date.day)

# file to open
file_str = ''

# tell functions to print for debugging
verbose = 1

while current_date <= end_date:    
    while current_hour < current_date + datetime.timedelta(days=1):
        # set search counter
        found = 0
        prev_day_count = 0
        most_recent_row = None

        # search files starting from current_date and 
        # continue if data point is not found for current_hour, current_date
        # end search after searching previous 4 days
        while found == 0 and prev_day_count <= 4:
            # Select rows that have CDATETIME created at most 4 days before or at same time as current_hour
            mask_is_valid_date = (available_dates_df['CDATETIME'] >= (current_hour - datetime.timedelta(days=4))) & (available_dates_df['CDATETIME'] <= current_hour)
            dt_mask = mask_is_valid_date

            # If such a row exists, select the row with closest CDATETIME to current_hour
            if available_dates_df[dt_mask].shape[0] > 0:
                most_recent_row = available_dates_df[dt_mask].iloc[-1]
                found = 1
            else:
                prev_day_count += 1
        
        # if we found a row, update idx_calls array to specify we will use:
        # TFLAG[tstep_idx] from the dispersion file named with CDATETIME to represent current_hour in our final IDX file
        if found:
            # the number of hours difference between 0th TFLAG and current_hour is tstep_idx
            tflag_0 = parse_tflag(most_recent_row['TFLAG'][0])
            tstep_idx = int((current_hour - tflag_0).total_seconds() / 3600)

            if verbose:
                print(f'Found data for current_hour {current_hour}: using CDATE={most_recent_row["CDATETIME"]}')
                print(f"tflag_0 = {tflag_0}, current_hour = {current_hour}, tstep_diff = {tstep_idx}")

            # update idx call array
            update_idx_calls(idx_calls, most_recent_row['CDATETIME'], tstep_idx)
        else:
            if verbose:
                print(f'WARNING: No available file found for current_hour {current_hour} (date: {current_date}). Skipping.')

        # move to next hour
        current_hour += datetime.timedelta(hours=1)

        print('~~~~~')

    # move to the next day
    current_date += datetime.timedelta(days=1)


~~~~~
~~~~~
~~~~~
Found data for current_hour 2021-02-28 03:00:00: using CDATE=2021-02-28 02:50:55
tflag_0 = 2021-02-28 03:00:00, current_hour = 2021-02-28 03:00:00, tstep_diff = 0
~~~~~
Found data for current_hour 2021-02-28 04:00:00: using CDATE=2021-02-28 02:50:55
tflag_0 = 2021-02-28 03:00:00, current_hour = 2021-02-28 04:00:00, tstep_diff = 1
~~~~~
Found data for current_hour 2021-02-28 05:00:00: using CDATE=2021-02-28 02:50:55
tflag_0 = 2021-02-28 03:00:00, current_hour = 2021-02-28 05:00:00, tstep_diff = 2
~~~~~
Found data for current_hour 2021-02-28 06:00:00: using CDATE=2021-02-28 02:50:55
tflag_0 = 2021-02-28 03:00:00, current_hour = 2021-02-28 06:00:00, tstep_diff = 3
~~~~~
Found data for current_hour 2021-02-28 07:00:00: using CDATE=2021-02-28 02:50:55
tflag_0 = 2021-02-28 03:00:00, current_hour = 2021-02-28 07:00:00, tstep_diff = 4
~~~~~
Found data for current_hour 2021-02-28 08:00:00: using CDATE=2021-02-28 02:50:55
tflag_0 = 2021-02-28 03:00:00, current_hour = 2021-02-28

In [34]:
%%capture captured_output
for c in idx_calls:
    print(c)

with open('idx_calls_v5.txt', 'w') as f:
    f.write(captured_output.stdout)

In [35]:
# save idx_calls to file
with open('idx_calls_v5.pkl', 'wb') as f:
    pickle.dump(idx_calls, f)

In [40]:
idx_calls[0]

['dispersion_2021059_025055.nc', datetime.datetime(2021, 2, 28, 3, 0), 0]

## Do conversion from netCDF files to IDX

In [36]:
print(f'len(idx_calls) = {len(idx_calls)}')
# idx_calls

len(idx_calls) = 45


In [41]:
# Create idx file of i'th dataset
# useful for dealing with fields that are not all the same size:
# https://github.com/sci-visus/OpenVisus/blob/master/Samples/jupyter/nasa_conversion_example.ipynb

# create OpenVisus field for the pm25 variable
f = Field('PM25', 'float32')

# create the idx file for this dataset using field f
# dims is maximum array size, we will resample data accordingly to fit this
# time is number of files * 24 (hours)
db = CreateIdx(url=idx_dir + '/firesmoke.idx', fields=[f], 
               dims=[int(max_grid['COL']), int(max_grid['ROW'])], time=[0, len(idx_calls) - 1, '%00000000d/'])

# to track what timestep we are on in IDX conversion
tstep = 0

# threshold to use to change small-enough resampled values to 0
thresh = 1e-15

for call in tqdm(idx_calls):
    # get instructions from call:
    # [file name to open, timestamp, TSTEP index to select]
    file_name = call[0]
    timestamp = call[1]
    tstep_index = call[2]
    # open the file with xarray
    ds = xr.open_dataset(f'{firesmoke_dir}/{file_name}')
    
    # Get the PM25 values, squeeze out empty axis
    file_vals = np.squeeze(ds['PM25'].values)
    
    # to decide if we need to resample or not
    resamp = ds.XORIG != max_grid['XORIG']
    
    # resample data if not already on max lat/lon grid
    if resamp:
        # Perform the interpolation
        file_vals_resamp = griddata(sml_tups, file_vals[tstep_index].flatten(), big_tups, method='cubic', fill_value=0)
        
        # Any values that are less than a given threshold, make it 0
        file_vals_resamp[file_vals_resamp < thresh] = 0
        
        # Reshape the result to match the new grid shape
        file_vals_resamp = file_vals_resamp.reshape((len(big_lat), len(big_lon)))
        # Write resampled values at hour h to timestep t and field f
        db.write(data=file_vals_resamp.astype(np.float32),field=f,time=tstep)
    else:
        # Write original values at hour h to timestep t and field f
        db.write(data=file_vals[tstep_index], field=f, time=tstep)

    # move to next timestep in IDX
    tstep = tstep + 1

100%|██████████| 45/45 [04:08<00:00,  5.52s/it]


In [42]:
# go to idx data directory
os.chdir(idx_dir)

In [43]:
# compress dataset
db.compressDataset(['zip'])